In [1]:
import requests
import csv 
from collections import Counter
from pandas import *

In [2]:
listoffiles=[]
listoffilmnames=[]

def csvgetter(user,password,*otherusernames):
    #first create a session 's'
    s=requests.session()
    #this is the data that is sent to login to icheckmovies
    login_data={'login[username]': user,'login[password]': password,'submit': 'login',}
    #here we send the login data, using the session created before
    s.post("https://www.icheckmovies.com/login/", data=login_data) 
    #once again, whithin the session we get the csv link and call it 'r'
    usernames_list=list(otherusernames)
    usernames_list.append(user)
    for item in usernames_list:
        url='https://www.icheckmovies.com/movies/checked/?user='+item+'&export'
        r=s.get(url)
        checklist=r.text
        checklist_encode = checklist.encode('utf-8')
        filename=item+'_checklist'+'.csv'
        file=open(filename, "w")
        file.write(checklist_encode)
        file.close() 
        listoffiles.append(filename)
    numberofusers=str((len(listoffiles)))
    print('There are '+numberofusers+' users in the database: '+'\n'+'\n'.join(listoffiles))
    
def listmaker(username):
    filmlist=[]
    toplist=[]
    mostchecked=[]
    csvname=username+"_checklist.csv"
    csv_reader = csv.reader(open(csvname, "rb"), delimiter=';')
    for row in csv_reader:
        filmlist.append(row[0])
        mostchecked.append(row[3])
        toplist.append(row[5])        
    filmlist.pop(0)
    mostchecked.pop(0)
    toplist.pop(0)
    compiled_dataframe={'title' : filmlist,
            'most_checked': mostchecked,
            'top_lists': toplist}
    userlist=DataFrame(compiled_dataframe)
    return(userlist)

def multiname(*usernames):
    for name in usernames:
        listmaker(name)
        listoffilmnames.append(filmlist)
        print(name)

In [3]:
username1='hermesmovies'
username2='Haessar'
username3='James88'
password=''

In [4]:
csvgetter(username1,password,username2,username3)

There are 3 users in the database: 
Haessar_checklist.csv
James88_checklist.csv
hermesmovies_checklist.csv


In [5]:
user1_list=listmaker(username1)
user2_list=listmaker(username2)
user3_list=listmaker(username3)
print("user1 is "+username1+", user2 is "+username2+" and user1 is "+username3+", user#_list is a dataframe of each users list")

user1 is hermesmovies, user2 is Haessar and user1 is James88, user#_list is a dataframe of each users list


Want to make a list that both james and will have seen that I haven't seen

In [37]:
#join will and james
combo_2_3=user2_list.append(user3_list)
#return duplicates
seenby_2_3=combo_2_3[combo_2_3.duplicated('title')==True] 

403


In [19]:
#this is working on a function that will do everything
def superfunction(usr1,pssword,*usrs):
    username1=usr1
    username2=usrs[0]
    username3=usrs[1]
    password=pssword
    csvgetter(username1,password,username2,username3)
    multiname(username1,username2,username3)